## Example: Multi-Objective Multi-Fidelity Hyperparameter Optimization

This notebook provides an example for AutoGluon's multi-objective HPO capabilities. Using the MNIST toy problem we show how to use the MOHyperbandScheduler to search for neural network hyperparameters which lead to networks with accurate and fast predictions.

The overall handling is very similar to the standard HyperbandScheduler. The constructor signature differs in two attributes:

1. **objectives**: This attribute is used to specify the objectives of interests. It expects a dictionary. The dictionary values  are used to indicate if a particular objective is meant to be maximized ("MAX") or minimized ("MIN"). 
2. **scalarization_options**: An additional dictionary can be provided to specify a scalarization technique and its parameters. By default "Random Weights" with 100 weight vectors is used.

In [ ]:
# imports
import time
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict 
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
import autogluon.core as ag

### Get MNIST data

In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X / 255.
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]
classes = np.unique(y)

### Define a training function that returns multiple objectives

In [ ]:
# Specify search space for MLPs
@ag.args(
    n_layers=ag.space.Int(1, 4),
    layer_1=ag.space.Int(2, 32),
    layer_2=ag.space.Int(2, 32),
    layer_3=ag.space.Int(2, 32),
    layer_4=ag.space.Int(2, 32),
    alpha=ag.space.Real(1e-6, 1e-1, log=True),
    learning_rate_init=ag.space.Real(1e-6, 1e-2, log=True),
    beta_1=ag.space.Real(0.001, 0.99, log=True),
    beta_2=ag.space.Real(0.001, 0.99, log=True),
    tol=ag.space.Real(1e-5, 1e-2, log=True),
    epochs=20)
def train_fn(args, reporter):
    
    # Initialize model with args
    hidden = [args.layer_1, args.layer_2, args.layer_3, args.layer_4][:args.n_layers]
    mlp = MLPClassifier(hidden_layer_sizes=hidden,
                        alpha=args.alpha,
                        learning_rate_init=args.learning_rate_init,
                        beta_1=args.beta_1,
                        beta_2=args.beta_2,
                        tol=args.tol,
                        verbose=False)

    # Train model iteratively
    for e in range(args.epochs):
        mlp.partial_fit(X_train, y_train, classes)
        train_score = mlp.score(X_train, y_train)
        test_score = mlp.score(X_test, y_test)
        start = time.time()
        mlp.predict(X_train)
        prediction_time = time.time() - start
        
        # Ensure to return all quantities of interest
        reporter(epoch=e+1, train_score=train_score, test_score=test_score, prediction_time=prediction_time)

### Run hyperparameter search

In [ ]:
# Objectives are specified as dictionary with indication if they should be 
# maximized or minimized
objectives = {
    "test_score": "MAX",
    "prediction_time": "MIN"
}

# The scalarization technqiue to be used and its parameters
# are provided as a dictionary. There are two different
# scalarization methods available
random_weights_options = {
    "algorithm": "random_weights",
    "num_weights": 100
}

parego_options = {
    "algorithm": "parego",
    "num_weights": 100,
    "rho": 0.05,
    "lambda": 3
}

scheduler = ag.scheduler.MOHyperbandScheduler(
    train_fn,
    resource={'num_cpus': 4, 'num_gpus': 0},
    num_trials=10,
    objectives=objectives,
    scalarization_options=random_weights_options,
    time_attr='epoch',
    grace_period=1,
    reduction_factor=3,
    type='stopping'
)
scheduler.run()
scheduler.join_jobs()

### Retrieve history for every objective

In [ ]:
histories = defaultdict(list)
for task_id, task_res in scheduler.training_history.items():
    for res_dict in task_res:
        for o in objectives:
            histories[o].append(res_dict[o])
         
    plt.ylabel(list(objectives.keys())[0])
    plt.xlabel(list(objectives.keys())[1])
    plt.title("MO Result")
    plt.scatter(histories["test_score"], histories["prediction_time"])